In [7]:
import pandas as pd
import geopandas as gpd
import fiona

In [9]:

# List all layers in the geodatabase
gdb_path = "Final_Project/Final_Project.gdb"
layers = fiona.listlayers(gdb_path)
print(layers)

# # Read a specific layer
df = gpd.read_file(gdb_path, layer="HouseFinal")

['HouseFinal_Withou_Statistics', 'OLS_Coefficient', 'OLS_Diagnostic', 'HouseFinal_SpatialOutlierDetection', 'HouseFinal_SpatialOutlierDetection1', 'tl_2019_12_tract_Project', 'Census_Tracts', 'Census_Tracts_XYTableToPoint', 'HouseFinal_SpatialOutlierDetection2', 'HouseFinal', 'HouseFinal_OrdinaryLeastSquares', 'HouseFinal_GWR', 'HouseFinal_HotSpots', 'test', 'test_label', 'Random_Sample', 'Random_Sample_GWR', 'Random_Sample_GWR2', 'Random_Sample_MGWR', 'Random_Sample_OrdinaryLeastSquares']


In [17]:
df = pd.read_csv('merged_data.csv')

In [14]:
df = pd.read_csv('/Users/nicholashorton/Documents/GIS Research Project/Advanced Search 7-23-2025.csv')

/var/folders/n2/fnwtrxs13n10mlfx7sftx0t00000gn/T/ipykernel_34239/3755842714.py:1: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/nicholashorton/Documents/GIS Research Project/Advanced Search 7-23-2025.csv')


In [22]:
df = pd.read_csv('2025.csv')
df[:20000].to_csv('2025_20k.csv', index=False)

/var/folders/n2/fnwtrxs13n10mlfx7sftx0t00000gn/T/ipykernel_34239/2049173357.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2025.csv')


In [5]:
sf = df[df['PROPERTY_USE']== '0110 Single Family Home']

# dropping unneeded columns
these columns are either not needed or do not apply to single family homes such as unit view or balcony/porch. these seem to apply to 
appartments or condos

In [ ]:
sf = sf.drop(columns=['SPECIAL_ASSESSMENT','JUST_LAND','JUST_BUILDING','JUST_EXTRA_FEATURES','VAL','OR_BKPG',
                      'FLOOR_NUM','BALCONY_PORCH','COMPLEX_VIEW','COOLING','UNIT_VIEW','PARKING_GARAGE'])

In [53]:
sf['PRICE'].describe()

count    2.841990e+05
mean     2.460057e+05
std      4.282612e+05
min      0.000000e+00
25%      1.000000e+02
50%      1.570000e+05
75%      3.360000e+05
max      3.224000e+07
Name: PRICE, dtype: float64

In [58]:
sf[sf['PRICE'] < 100]['REA_CD_DSCR'].value_counts()

REA_CD_DSCR
11-Corrective, QC or Tax Deed or Minimum Doc Stamps                                                                                                    31717
78-Certificate of Title\t                                                                                                                                187
19-Trustees in bankruptcy, executors, guardian, recv                                                                                                      53
30-Affiliated parties: fam, corp, bus, landlord/ten                                                                                                       39
70-Deed in lieu of Foreclosure\t                                                                                                                          20
18-Gov: Fed, St, local, muni, educ, shrf, court,TIITF                                                                                                     12
12-Deeds to or from financial institutions    

In [55]:
sf[sf['PRICE'] < 100]['QU_FLG'].value_counts()

QU_FLG
U    32087
Q        1
Name: count, dtype: int64

from what I can find, the QU_FLG field tells you whether the sale is considered a “qualified” (valid) market transaction. and all of these houses that have a sale price of $100 or lower
are all unqualified execpt for one which i will also drop. 

In [82]:
sf = sf[sf['QU_FLG'] == 'Q']
sf = sf[sf['PRICE'] > 100]

In [61]:
sf['PRICE'].describe()

count    1.401300e+05
mean     4.228764e+05
std      4.454498e+05
min      2.000000e+03
25%      2.150000e+05
50%      3.190000e+05
75%      4.750000e+05
max      1.650000e+07
Name: PRICE, dtype: float64

In [70]:
sf['FRONTAGE'].value_counts()

FRONTAGE
Residential        8152
Canal/River        7421
Pond               5926
Intracoastal       2091
Park/Cons/Pres     1861
Golf Course        1657
Street             1504
Gulf                529
Tampa Bay           485
Lake                268
Commercial          103
Parking               4
Recreation Area       2
Name: count, dtype: int64

In [69]:
sf['VIEWS'].value_counts()

VIEWS
Park/Cons/Pres     3213
Pond                450
Golf Course         326
Canal/River         266
Intracoastal         66
Gulf                 62
Lake                 25
Tampa Bay            22
Ltd Intrcoastal       9
Ltd Gulf              1
Name: count, dtype: int64

for frontage and views I am going to fill these in with either no view or no frontage.

In [85]:
sf['FRONTAGE'] = sf['FRONTAGE'].fillna('No Frontage')
sf['VIEWS'] = sf['VIEWS'].fillna('No View')

In [86]:
# drop remaining null values
sf = sf.dropna()

# map yes/no fields to 1 and 0

In [84]:
sf.SEAWALL.value_counts()

SEAWALL
No     130277
Yes      9445
Name: count, dtype: int64

In [78]:
sf['SEAWALL'] = sf['SEAWALL'].map({'Yes': 1, 'No': 0})
sf['DLHL_YN'] = sf['DLHL_YN'].map({'Y': 1, 'N': 0})
sf['POOL'] = sf['POOL'].map({'Y': 1, 'N': 0})
sf['WATERFRONT_YN'] = sf['WATERFRONT_YN'].map({'Y': 1, 'N': 0})
sf['CLUBHOUSE'] = sf['CLUBHOUSE'].map({'Yes': 1, 'No': 0})
sf['MARINA'] = sf['MARINA'].map({'Yes': 1, 'No': 0})
sf['TENNIS'] = sf['TENNIS'].map({'Yes': 1, 'No': 0})

/var/folders/n2/fnwtrxs13n10mlfx7sftx0t00000gn/T/ipykernel_18213/400538200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf['SEAWALL'] = sf['SEAWALL'].map({'Yes': 1, 'No': 0})
/var/folders/n2/fnwtrxs13n10mlfx7sftx0t00000gn/T/ipykernel_18213/400538200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf['DLHL_YN'] = sf['DLHL_YN'].map({'Y': 1, 'N': 0})
/var/folders/n2/fnwtrxs13n10mlfx7sftx0t00000gn/T/ipykernel_18213/400538200.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl